In [1]:
# python 3.9.7

import pandas as pd 
import duckdb 
import numpy as np

##### Notebook Functions

In [2]:
def create_clients_tbl(con):
#   Function name: create_clients_tbl
#   Description: This function is used to join processed pandas dataframes with client information
#   Parameters: con
#        con(DuckDB connection): The DuckDB access point to create DuckDB SQL queries
#   Return values: df
#        df(pandas dataframe): The output df with combined clients tables

    df = pd.read_csv("../output_tables/clients_v1.csv")
    df2 = pd.read_csv("../output_tables/clients_v2.csv")
    df3 = pd.read_csv("../output_tables/clients_v3.csv")
    
    # Join all three tables together
    df = con.execute(f"""SELECT client_id, company_name, tier, status, active_flag, currency, cli_join_dt FROM
                        (WITH t1 AS (SELECT df.*, df2.* EXCLUDE(client_id, company_name, cli_join_dt) FROM df 
                        JOIN df2 ON df.client_id = df2.client_id) 
                        SELECT t1.*, df3.* EXCLUDE(client_id, company_name, cli_join_dt) FROM t1 
                        JOIN df3 ON t1.client_id = df3.client_id)
                        """).fetchdf()
    
    # Take the higher tier company_name whenever a company_name has 2 or entries of different tiers
    df = con.execute("""SELECT * EXCLUDE(tier_rk, rk) FROM
                    (WITH temp AS (SELECT *, 
                    CASE WHEN tier = 'GOLD' THEN 3  
                    WHEN tier = 'SILVER' THEN 2 
                    WHEN tier = 'BRONZE' THEN 1 
                    ELSE 0 END as tier_rk FROM df)
                    SELECT *, RANK() OVER (PARTITION BY company_name, client_id ORDER BY tier_rk DESC) as rk FROM temp)
                    WHERE rk = 1""").fetchdf()
    
    # Fill NULL tier as BRONZE as long as the company is active, otherwise keep as NULL
    df['tier'] = np.where(((df.tier.isnull()) & (df.active_flag == 'Y')), 'BRONZE', df.tier)
    
    # Fill NULL status as INACTIVE as long as tier is NULL
    df['status'] = np.where(((df.tier.isnull()) & (df.status.isnull())), 'INACTIVE', df.status)

    # Fill NULL active_flag as N as long as tier is NULL
    df['active_flag'] = np.where(((df.tier.isnull()) & (df.active_flag.isnull())), 'N', df.active_flag)
    
    return df

In [3]:
def create_invoices_tbl(con):
#   Function name: create_invoices_tbl
#   Description: This function is used to join processed pandas dataframes with invoice information
#   Parameters: con
#        con(DuckDB connection): The DuckDB access point to create DuckDB SQL queries
#   Return values: df
#        df(pandas dataframe): The output df with combined invoices tables

    df = pd.read_csv("../output_tables/invoices_v1.csv")
    df2 = pd.read_csv("../output_tables/invoices_v2.csv")
    df3 = pd.read_csv("../output_tables/invoices_v3.csv")
    
    # Join all three tables together
    df = con.execute(f"""SELECT invoice_id, invoice_date, client_id, company_name, shipment_type, subtotal, tax, total, currency FROM
                        (WITH t1 AS (SELECT df.*, df2.* EXCLUDE (invoice_id, invoice_date, shipment_type, client_id) FROM df JOIN df2 ON df.invoice_id = df2.invoice_id) 
                        SELECT t1.*, df3.* EXCLUDE (invoice_id, invoice_date, shipment_type) FROM t1 JOIN df3 ON t1.invoice_id = df3.invoice_id)
                        """).fetchdf()
    
    return df

##### Put together tables

In [4]:
def construct_final_tbl(con):
#   Function name: construct_final_tbl
#   Description: This function is used to join the clients and invoices tables
#   Parameters: con
#        con(DuckDB connection): The DuckDB access point to create DuckDB SQL queries
#   Return values: df
#        df(pandas dataframe): The output df with combined tables to use for analysis queries

    df_cli = create_clients_tbl(con)
    df_inv = create_invoices_tbl(con)

    df = con.execute("""SELECT df_cli.client_id, df_cli.company_name, invoice_id, invoice_date, total as invoice_amt, df_cli.currency, shipment_type FROM df_cli JOIN df_inv 
                        ON df_cli.client_id = df_inv.client_id AND UPPER(df_cli.company_name) = TRIM(df_inv.company_name)""").fetchdf()
    display(df)
    
    return df
    
con = duckdb.connect(database=":memory:")
df = construct_final_tbl(con)

,client_id,company_name,invoice_id,invoice_date,invoice_amt,currency,shipment_type
0,C10456,Blue Partners,INV-ITBHR2P,2025-05-22,5654.16,USD,2DAY
1,C16655,hooli co,INV-LWC0YM6,2024-04-19,3165.79,USD,GROUND
2,C63096,Initech Freight,INV-7TDN2MF,2025-07-28,3326.33,USD,GROUND
3,C15499,Zenith Holdings,INV-N9Z2HI7,2025-05-30,3514.01,USD,GROUND
4,C93089,Blue Supply,INV-M8ZSF57,2024-07-14,3702.47,USD,GROUND
...,...,...,...,...,...,...,...
11995,C63096,Initech Freight,INV-5LPB2GE,2025-12-06,3510.99,USD,GROUND
11996,C86029,Acme Industries,INV-CE1RRD5,2025-09-05,15561.43,USD,EXPRESS
11997,C57382,Vector Logistics,INV-SU0IUIL,2025-01-26,2700.09,USD,GROUND
11998,C17172,Apex Freight,INV-FH9UUJM,2025-05-19,42958.88,USD,FREIGHT


# Analysis Questions

1. Basic: Which top 5 clients have the largest total invoice amounts outstanding?

In [5]:
answ = con.execute("""SELECT client_id, company_name, ROUND(SUM(invoice_amt), 2) as sum FROM df 
                      GROUP BY client_id, company_name 
                      ORDER BY sum DESC 
                      LIMIT 5""").fetchdf()
answ.to_csv("../output_tables/answer1.csv", index=False)
print('Answer to question 1')
display(answ)

Answer to question 1


,client_id,company_name,sum
0,C25055,Stark Partners,2244869.53
1,C94736,Red Logistics,2218130.71
2,C14175,Wayne Group,2151906.65
3,C03366,Umbrella Industries,2108671.57
4,C77726,Nimbus Holdings,2096498.84


2. Intermediate: Show the month-over-month invoice growth per client for 2024–2025.

In [6]:
answ = con.execute("""WITH yr_mo AS (SELECT *, strftime(CAST(invoice_date AS DATE), '%Y-%m') AS year_month FROM df 
                       WHERE invoice_date  BETWEEN '2024-01-01' AND '2025-12-30')
                       SELECT company_name, year_month, ROUND(SUM(invoice_amt), 2) as sum FROM yr_mo
                       GROUP BY company_name, year_month
                       ORDER BY company_name, year_month""").fetchdf()
answ.to_csv("../output_tables/answer2.csv", index=False)
print('Answer to question 2')
display(answ)

Answer to question 2


,company_name,year_month,sum
0,Acme Industries,2024-01,29710.94
1,Acme Industries,2024-02,72471.83
2,Acme Industries,2024-03,82602.44
3,Acme Industries,2024-04,32468.28
4,Acme Industries,2024-05,15269.11
...,...,...,...
1290,hooli co,2025-08,79274.49
1291,hooli co,2025-09,101361.63
1292,hooli co,2025-10,56309.51
1293,hooli co,2025-11,78504.04


3. Discount Scenario: Show total costs for each client, if discounts were applied:
 - a.) 20% off GROUND
     - Then, who are the new top 5 spenders?
 - b.) 30% off FREIGHT
     - Then, who are the new top 5 spenders?
 - c.) 50% off 2 DAY
     - Then, who are the new top 5 spenders?


In [7]:
def answer_q_3(part):
    if part == 'a':
        disc = 0.2
        ship_type = 'GROUND'
    elif part == 'b':
        disc = 0.3
        ship_type = 'FREIGHT'
    elif part == 'c':
        disc = 0.5
        ship_type = '2DAY'
    df_temp = con.execute(f"""SELECT *, 
                           CASE 
                           WHEN shipment_type = 'GROUND' THEN 1
                           WHEN shipment_type = '2DAY' THEN 5
                           WHEN shipment_type = 'EXPRESS' THEN 10
                           WHEN shipment_type = 'FREIGHT' THEN 20 
                           END as shipment_price,
                           invoice_amt / shipment_price as unit,
                           CASE WHEN shipment_type = '{ship_type}' THEN shipment_price - (shipment_price * {disc}) 
                           ELSE shipment_price END as new_price,
                           unit * new_price as new_total_amt
                           FROM df""").fetchdf()
    answ = con.execute("""SELECT client_id, company_name, ROUND(SUM(new_total_amt), 2) as sum FROM df_temp 
                          GROUP BY client_id, company_name 
                          ORDER BY sum DESC 
                          LIMIT 5""").fetchdf()
    answ.to_csv(f"../output_tables/answer3part{part}.csv", index=False)
    print(f'Answer to Question 3 part {part}')
    display(answ)
    
answer_q_3('a')
answer_q_3('b')
answer_q_3('c')

Answer to Question 3 part a


,client_id,company_name,sum
0,C25055,Stark Partners,2186923.01
1,C94736,Red Logistics,2145638.36
2,C14175,Wayne Group,2095712.67
3,C03366,Umbrella Industries,2046015.82
4,C77726,Nimbus Holdings,2028150.87


Answer to Question 3 part b


,client_id,company_name,sum
0,C94736,Red Logistics,1985352.74
1,C25055,Stark Partners,1958102.47
2,C14175,Wayne Group,1867187.99
3,C77726,Nimbus Holdings,1862845.99
4,C03366,Umbrella Industries,1852070.78


Answer to Question 3 part c


,client_id,company_name,sum
0,C25055,Stark Partners,2152194.96
1,C94736,Red Logistics,2072278.81
2,C14175,Wayne Group,2039389.28
3,C03366,Umbrella Industries,2003502.09
4,C60889,Wonka LLC,1995832.07


4. Reclassification Scenario: Suppose all “EXPRESS” shipments were instead billed as “GROUND” (lower cost).
 - a.) What is the total cost savings opportunity per client?
 - b.) Which clients have > 50% savings?
 - c.) Which clients have > $500k savings?


In [8]:
def answer_q_4(part):
    df_temp = con.execute(f"""SELECT *, 
                           CASE 
                           WHEN shipment_type = 'GROUND' THEN 1
                           WHEN shipment_type = '2DAY' THEN 5
                           WHEN shipment_type = 'EXPRESS' THEN 10
                           WHEN shipment_type = 'FREIGHT' THEN 20 
                           END as shipment_price,
                           CASE WHEN shipment_type = 'EXPRESS' THEN 1 ELSE shipment_price END scenario_price,
                           invoice_amt / shipment_price as unit
                           FROM df""").fetchdf()
    df_temp = con.execute("""WITH temp AS 
                          (SELECT *, unit * scenario_price as new_price, ROUND(invoice_amt - new_price, 2) as savings 
                          FROM df_temp)

                          SELECT client_id, company_name, ROUND(SUM(invoice_amt), 2) as total_orig_amt, 
                          ROUND(SUM(new_price), 2) as total_new_price, ROUND(SUM(savings), 2) as total_savings FROM temp
                          GROUP BY client_id, company_name""").fetchdf()
    if part == 'a':
        answ = con.execute("""SELECT * FROM df_temp 
                              ORDER BY total_savings DESC""").fetchdf()
    elif part == 'b':
        answ = con.execute("""SELECT * EXCLUDE(total_orig_amt, total_new_price), 
                              (total_orig_amt - total_new_price) / total_orig_amt as savings FROM df_temp 
                              WHERE savings > 0.5
                              ORDER BY total_savings DESC""").fetchdf()
    elif part == 'c':
        answ = con.execute("""SELECT * EXCLUDE(total_orig_amt, total_new_price) FROM df_temp 
                              WHERE total_savings > 500000
                              ORDER BY total_savings DESC""").fetchdf()
    answ.to_csv(f"../output_tables/answer4part{part}.csv", index=False)
    print(f'Answer to Question 4 part {part}')
    display(answ)
    
answer_q_4('a')
answer_q_4('b')
answer_q_4('c')

Answer to Question 4 part a


,client_id,company_name,total_orig_amt,total_new_price,total_savings
0,C82971,Massive Co,1741866.19,928198.01,813668.19
1,C83948,Wayne Holdings,1631001.97,889670.82,741331.16
2,C99892,Wayne Freight,2062937.31,1321909.84,741027.47
3,C25055,Stark Partners,2244869.53,1512361.71,732507.86
4,C94736,Red Logistics,2218130.71,1508896.02,709234.72
5,C99808,Blue Co,1655126.40,950648.56,704477.88
6,C92842,Tyrell Industries,1962423.79,1271400.76,691023.05
7,C15499,Zenith Holdings,1856326.82,1166650.56,689676.26
8,C63250,Vertex Supply,2018888.92,1332444.94,686443.99
9,C77726,Nimbus Holdings,2096498.84,1420508.73,675990.14


Answer to Question 4 part b


,client_id,company_name,total_savings,savings


Answer to Question 4 part c


,client_id,company_name,total_savings
0,C82971,Massive Co,813668.19
1,C83948,Wayne Holdings,741331.16
2,C99892,Wayne Freight,741027.47
3,C25055,Stark Partners,732507.86
4,C94736,Red Logistics,709234.72
5,C99808,Blue Co,704477.88
6,C92842,Tyrell Industries,691023.05
7,C15499,Zenith Holdings,689676.26
8,C63250,Vertex Supply,686443.99
9,C77726,Nimbus Holdings,675990.14


In [9]:
con.close()